# Update Spec Name

This dataset was created with a spec name "default" which is reserved for the OpenFF default model chemistry: `b3lyp-d3bj/dzvp`. We will update the spec name to be the method and basis joined with a slash.

In [1]:
import os
import sys

from qcportal import PortalClient

In [2]:
ADDRESS = "https://api.qcarchive.molssi.org:443/"
qc_client = PortalClient(
    ADDRESS, 
    username=os.environ['QCARCHIVE_USER'],
    password=os.environ['QCARCHIVE_PASSWORD'],
    cache_dir=".",
)

ds = qc_client.get_dataset("torsiondrive", "OpenFF Theory Benchmarking Set B3LYP-D3BJ 6-31+Gss")
print(f"Spec Names: {ds.specification_names}")
for spec_name, spec in ds.specifications.items():
    print(f"Name: {spec_name}")
    if spec_name == "default":
        old_name = "default"
        old_spec = spec.specification
        spec = spec.specification.optimization_specification.qc_specification
        print(f"    Method: {spec.method}")
        print(f"    Basis: {spec.basis}")
        new_name = f"{spec.method.lower()}/{spec.basis.lower()}"
        if new_name == "b3lyp-d3bj/dzvp":
            sys.exit("This is the default model chemistry!")
        else:
            print(f"    New Name will be: {new_name}")

Spec Names: ['default']
Name: default
    Method: b3lyp-d3bj
    Basis: 6-31+g**
    New Name will be: b3lyp-d3bj/6-31+g**


In [3]:
old_name, old_spec, new_name

('default',
 TorsiondriveSpecification(program='torsiondrive', optimization_specification=OptimizationSpecification(program='geometric', qc_specification=QCSpecification(program='psi4', driver=<SinglepointDriver.deferred: 'deferred'>, method='b3lyp-d3bj', basis='6-31+g**', keywords={'maxiter': 200, 'scf_properties': ['dipole', 'quadrupole', 'wiberg_lowdin_indices', 'mayer_indices']}, protocols=AtomicResultProtocols(wavefunction=<WavefunctionProtocolEnum.none: 'none'>, stdout=True, error_correction=ErrorCorrectionProtocol(default_policy=True, policies=None), native_files=<NativeFilesProtocolEnum.none: 'none'>)), keywords={'tmax': 0.3, 'check': 0, 'qccnv': True, 'reset': True, 'trust': 0.1, 'molcnv': False, 'enforce': 0.1, 'epsilon': 0, 'maxiter': 300, 'coordsys': 'tric', 'convergence_set': 'gau'}, protocols=OptimizationProtocols(trajectory=<TrajectoryProtocolEnum.all: 'all'>)), keywords=TorsiondriveKeywords(dihedrals=[], grid_spacing=[], dihedral_ranges=None, energy_decrease_thresh=None

In [4]:
ds.add_specification(name=new_name, specification=old_spec)
ds.submit()

InsertCountsMetadata(n_inserted=0, n_existing=36, error_description=None, errors=[])

In [5]:
print(ds.status_table())

rec_spec_old = set([rec.id for _, _, rec in ds.iterate_records(specification_names=[old_name])])
rec_spec_new = set([rec.id for _, _, rec in ds.iterate_records(specification_names=[new_name])])
print(f"\nSpecifications represent the same records? {rec_spec_old == rec_spec_new}")

      specification    complete
-------------------  ----------
b3lyp-d3bj/6-31+g**          36
            default          36

Specifications represent the same records? True


In [6]:
ds.delete_specification(old_name, delete_records=False)

DeleteMetadata(error_description=None, errors=[], deleted_idx=[0], n_children_deleted=0)

In [7]:
print(ds.status_table())

      specification    complete
-------------------  ----------
b3lyp-d3bj/6-31+g**          36
